In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "Data/Updated_Scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")] 

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score 

In [7]:
def read_stats (soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df
    

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container") [0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0] 
    return season

In [9]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1, :]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format ="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 1 == 0:
        print(f"{len(games)} / {len(box_scores)}")
    
    



1 / 13
2 / 13
3 / 13
4 / 13
5 / 13
6 / 13
7 / 13
8 / 13
9 / 13
10 / 13
11 / 13
12 / 13
13 / 13


In [10]:
games_df = pd.concat(games, ignore_index=True)

In [11]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,34.0,80.0,0.425,8.0,27.0,0.296,21.0,31.0,...,23.3,34.1,156.0,109.0,PHI,104,1,2024,2024-02-23,False
1,240.0,240.0,40.0,90.0,0.444,12.0,29.0,0.414,12.0,15.0,...,27.3,26.6,148.0,118.0,CLE,97,0,2024,2024-02-23,True
2,240.0,240.0,46.0,81.0,0.568,15.0,35.0,0.429,9.0,12.0,...,42.0,40.9,160.0,147.0,NYK,102,1,2024,2024-02-24,True
3,240.0,240.0,37.0,82.0,0.451,13.0,38.0,0.342,15.0,16.0,...,25.0,37.3,278.0,135.0,BOS,116,0,2024,2024-02-24,False
4,240.0,240.0,50.0,102.0,0.490,15.0,39.0,0.385,8.0,8.0,...,22.4,39.2,158.0,128.0,ATL,121,1,2024,2024-02-23,True
5,240.0,240.0,46.0,106.0,0.434,12.0,39.0,0.308,17.0,24.0,...,33.3,25.8,139.0,124.0,TOR,123,0,2024,2024-02-23,False
6,240.0,240.0,38.0,82.0,0.463,13.0,29.0,0.448,17.0,25.0,...,18.8,32.2,150.0,113.0,NOP,95,1,2024,2024-02-23,True
7,240.0,240.0,39.0,104.0,0.375,7.0,32.0,0.219,10.0,13.0,...,29.4,33.7,241.0,109.0,MIA,106,0,2024,2024-02-23,False
8,240.0,240.0,43.0,87.0,0.494,12.0,24.0,0.500,14.0,21.0,...,42.9,34.4,168.0,130.0,DET,109,1,2024,2024-02-24,True
9,240.0,240.0,43.0,85.0,0.506,14.0,31.0,0.452,9.0,13.0,...,30.0,32.5,174.0,126.0,ORL,112,0,2024,2024-02-24,False


In [12]:
games_df.to_csv('updated_nba_boxscores.csv')